In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import shutil
from sklearn.model_selection import train_test_split
import glob
import torch

In [ ]:
PROJECT_DIRECTORY = '/content/drive/MyDrive/plexor_ai_engineer/'
VIDEO_DIRECTORY = os.path.join(PROJECT_DIRECTORY, "videos")
print(VIDEO_DIRECTORY)
FRAMES_DIRECTORY = os.path.join(PROJECT_DIRECTORY, "frames")
print(FRAMES_DIRECTORY)
DATASET_DRIECTORY = os.path.join(PROJECT_DIRECTORY, "dataset")
print(DATASET_DRIECTORY)
UPLOADED_LABELS = os.path.join(PROJECT_DIRECTORY, "labels_generated")
print(UPLOADED_LABELS)

/content/drive/MyDrive/plexor_ai_engineer/videos
/content/drive/MyDrive/plexor_ai_engineer/frames
/content/drive/MyDrive/plexor_ai_engineer/dataset
/content/drive/MyDrive/plexor_ai_engineer/labels_generated


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device: {device}")

device: cuda


In [ ]:
label_files = glob.glob(f"{UPLOADED_LABELS}/*.txt")
if len(label_files) == 0:
  print("Labels not found")
  print(f"Upload to {UPLOADED_LABELS}")
else:
  print("Labels found")
  print(f"Total: {len(label_files)}")

Labels found
Total: 185


In [ ]:
image_label_pairs = []
for label in label_files:
  label_name = os.path.basename(label)
  image_name = label_name.replace('.txt', '.jpg')

  for folder in os.listdir(FRAMES_DIRECTORY):
    path = os.path.join(FRAMES_DIRECTORY, folder, image_name)
    if os.path.exists(path):
      image_label_pairs.append((path, label))
      break

total_pairs = len(image_label_pairs)
print(f"Matched {total_pairs}")

Matched 185


In [ ]:
train_set, val_set = train_test_split(image_label_pairs, test_size=0.2, random_state=42)
print(f"Train size: {len(train_set)}")
print(f"Val size: {len(val_set)}")

Train size: 148
Val size: 37


In [ ]:
train_images = f"{DATASET_DRIECTORY}/images/train/"
train_labels = f"{DATASET_DRIECTORY}/labels/train/"

val_images = f"{DATASET_DRIECTORY}/images/val/"
val_labels = f"{DATASET_DRIECTORY}/labels/val/"

for image, label in train_set:
  shutil.copy(image, train_images)
  shutil.copy(label, train_labels)

for image, label in val_set:
  shutil.copy(image, val_images)
  shutil.copy(label, val_labels)

In [ ]:
yaml = f"""
path: {DATASET_DRIECTORY}
train: images/train
val: images/val
nc: 2
names: ['person', 'item']
"""

with open(f"{DATASET_DRIECTORY}/data.yaml", "w") as file:
  file.write(yaml.strip())

In [ ]:
from ultralytics import YOLO

data_path = f"{DATASET_DRIECTORY}/data.yaml"
versions_path = f"{PROJECT_DIRECTORY}/versions"
model = YOLO("yolov8s.pt")

results = model.train(
    data = data_path,
    epochs=100,
    imgsz=640,
    batch=8,
    device=device,
    project=versions_path,
    name='plexor_v2'

)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.223 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/plexor_ai_engineer/dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, 

In [ ]:
versions = glob.glob(f"{PROJECT_DIRECTORY}/versions/plexor_v**")
latest_version = sorted(versions)[-1]

os.makedirs(f"{PROJECT_DIRECTORY}/weights", exist_ok=True)
shutil.copy(
    f"{latest_version}/weights/best.pt",
    f"{PROJECT_DIRECTORY}/weights/best.pt"
)

print(f"Weights saved from {latest_version}")

Weights saved from /content/drive/MyDrive/plexor_ai_engineer//versions/plexor_v2
